# Compute conformer energies for a small molecule

This notebook illustrates reading conformers of a molecule from an SDF file and computation of vacuum conformer energies using a SMIRNOFF force field.

Note that absolute vacuum potential energies can be sensitive to small changes in partial charge, for example due to using OpenEye or AmberTools to generate AM1-BCC charges. However, in our experience, _relative_ conformer energies are fairly consistent between AM1-BCC implementations.

Note also that the Open Force Field Toolkit produces deterministic charges that do not depend on the input conformation of parameterized molecules. See the [FAQ](https://open-forcefield-toolkit.readthedocs.io/en/latest/faq.html#the-partial-charges-generated-by-the-toolkit-don-t-seem-to-depend-on-the-molecule-s-conformation-is-this-a-bug) for more information.

In [1]:
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.utils import RDKitToolkitWrapper, get_data_file_path
from rdkit.Chem import rdMolAlign
import numpy as np
from simtk import openmm, unit
import shutil

In [2]:
# If using a OFF Toolkit version before 0.7.0, loading SDFs through RDKit and OpenEye may provide 
# different behavior in some cases. So, here we force loading through RDKit to ensure the correct behavior
rdktkw = RDKitToolkitWrapper()

# Locate molecule in OpenFF Toolkit package data and copy to local directory
orig_path = get_data_file_path('molecules/ruxolitinib_conformers.sdf')
shutil.copy(orig_path, '.')

# Load in the molecule and its conformers.
# Note that all conformers of the same molecule are loaded as separate Molecule objects
loaded_molecules = Molecule.from_file('ruxolitinib_conformers.sdf', toolkit_registry=rdktkw)

# The logic below only works for lists of molecules, so if a
# single molecule was loaded, cast it to list
if type(loaded_molecules) is not list:
    loaded_molecules = [loaded_molecules]

# Collatate all conformers of the same molecule
# NOTE: This isn't necessary if you have already loaded or created multi-conformer molecules;
# it is just needed because our SDF reader does not automatically collapse conformers.
molecules = [loaded_molecules[0]]
for molecule in loaded_molecules[1:]:
    if molecule == molecules[-1]:
        for conformer in molecule.conformers:
            molecules[-1].add_conformer(conformer)
    else:
        molecules.append(molecule)

n_molecules = len(molecules)
n_conformers = sum([mol.n_conformers for mol in molecules])
print(f'{n_molecules} unique molecule(s) loaded, with {n_conformers} total conformers')

1 unique molecule(s) loaded, with 10 total conformers


In [3]:
# Load the openff-1.1.0 force field appropriate for vacuum calculations (without constraints)
from openff.toolkit.typing.engines.smirnoff import ForceField
forcefield = ForceField('openff_unconstrained-1.1.0.offxml')

In [4]:
# Loop over molecules and minimize each conformer
for molecule in molecules:
    # If the molecule doesn't have a name, set mol.name to be the hill formula
    if molecule.name == '':
        molecule.name = Topology._networkx_to_hill_formula(molecule.to_networkx())
    print('%s : %d conformers' % (molecule.name, molecule.n_conformers))
    # Make a temporary copy of the molecule that we can update for each minimization
    mol_copy = Molecule(molecule)
    # Make an OpenFF Topology so we can parameterize the system
    off_top = molecule.to_topology()
    print(f"Parametrizing {molecule.name} (may take a moment to calculate charges)")
    system = forcefield.create_openmm_system(off_top)
    # Use OpenMM to compute initial and minimized energy for all conformers
    integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
    platform = openmm.Platform.getPlatformByName('Reference')
    omm_top = off_top.to_openmm()
    simulation = openmm.app.Simulation(omm_top, system, integrator, platform)
    
    # Print text header
    print('Conformer         Initial PE        Minimized PE        RMS between initial and minimized conformer')
    output = [['Conformer','Initial PE (kcal/mol)','Minimized PE (kcal/mol)','RMS between initial and minimized conformer (Angstrom)']]
    for conformer_index, conformer in enumerate(molecule.conformers):
        simulation.context.setPositions(conformer)
        orig_potential = simulation.context.getState(getEnergy=True).getPotentialEnergy()
        simulation.minimizeEnergy()
        min_state = simulation.context.getState(getEnergy=True, getPositions=True)
        min_potential = min_state.getPotentialEnergy()
        
        # Calculate the RMSD between the initial and minimized conformer
        min_coords = min_state.getPositions()
        min_coords = np.array([ [atom.x, atom.y, atom.z] for atom in min_coords]) * unit.nanometer
        mol_copy._conformers = None
        mol_copy.add_conformer(conformer)
        mol_copy.add_conformer(min_coords)
        rdmol = mol_copy.to_rdkit()
        rmslist = []
        rdMolAlign.AlignMolConformers(rdmol, RMSlist=rmslist)
        minimization_rms = rmslist[0]

        # Save the minimized conformer to file
        mol_copy._conformers = None
        mol_copy.add_conformer(min_coords)
        mol_copy.to_file(f'{molecule.name}_conf{conformer_index+1}_minimized.sdf', file_format='sdf')
        print('%5d / %5d : %8.3f kcal/mol %8.3f kcal/mol  %8.3f Angstroms' % (conformer_index+1, molecule.n_conformers, orig_potential/unit.kilocalories_per_mole, min_potential/unit.kilocalories_per_mole, minimization_rms))
        output.append([str(conformer_index+1), 
                       f'{orig_potential/unit.kilocalories_per_mole:.3f}', 
                       f'{min_potential/unit.kilocalories_per_mole:.3f}', 
                       f'{minimization_rms:.3f}'])
    # Write the results out to CSV
    with open(f'{molecule.name}.csv', 'w') as of:
        for line in output:
            of.write(','.join(line)+'\n')
    # Clean up OpenMM Simulation
    del simulation, integrator

ruxolitinib : 10 conformers
Parametrizing ruxolitinib (may take a moment to calculate charges)


Conformer         Initial PE        Minimized PE        RMS between initial and minimized conformer
    1 /    10 :   60.113 kcal/mol   31.212 kcal/mol     0.728 Angstroms
    2 /    10 :   69.741 kcal/mol   32.679 kcal/mol     0.511 Angstroms
    3 /    10 :  105.413 kcal/mol   32.485 kcal/mol     0.931 Angstroms
    4 /    10 :   66.266 kcal/mol   32.874 kcal/mol     0.644 Angstroms
    5 /    10 :   67.877 kcal/mol   33.932 kcal/mol     0.970 Angstroms
    6 /    10 :   69.128 kcal/mol   31.004 kcal/mol     0.601 Angstroms
    7 /    10 :   68.287 kcal/mol   32.997 kcal/mol     1.151 Angstroms
    8 /    10 :   58.255 kcal/mol   31.824 kcal/mol     0.767 Angstroms
    9 /    10 :   64.240 kcal/mol   31.260 kcal/mol     1.057 Angstroms
   10 /    10 :   66.048 kcal/mol   30.187 kcal/mol     0.616 Angstroms
